<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llm/ai21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI21

This notebook shows how to use AI21's newest model - Jamba-Instruct. Along with support for the previous model family - Jurassic (J2-Mid and J2-Ultra).

By default - jamba-instruct model is used. If you want to use the Jurassic models, you can specify the model name as "j2-mid" or "j2-ultra".

## Basic Usage

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-llms-ai21

In [ ]:
!pip install llama-index

## Setting API Key

When creating the AI21 instance, you can pass the API key as a parameter or by default, it will be consumed from the environment variable `AI21_API_KEY`.

In [ ]:
import os
from llama_index.llms.ai21 import AI21

api_key = "Your-api-key"
os.environ["AI21_API_KEY"] = api_key

llm = AI21()

# Or
llm = AI21(api_key=api_key)

Paul Graham is a computer scientist, entrepreneur, and author. He is best known as the co-founder of Y Combinator, a venture capital firm that provides seed funding to early-stage startups. He is also a well-known essayist and writer on various topics such as programming, startups, and technology. He has written several influential essays on these topics, which have been widely read and discussed in the tech community.
Paul Graham is a computer scientist, entrepreneur, and essayist known for his work in the field of computer programming and his contributions to the startup ecosystem. He is the co-founder of Viaweb, which was later sold to Yahoo! and became Yahoo! Store. Graham is also recognized for his essays on various topics related to technology, startups, and the nature of innovation, which are widely read and discussed in the tech community. He co-founded Y Combinator, one of the most prominent startup accelerators in the world, which has helped launch over a thousand startups, i

#### Call `chat` with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.ai21 import AI21

messages = [
    ChatMessage(role="user", content="hello there"),
    ChatMessage(
        role="assistant", content="Arrrr, matey! How can I help ye today?"
    ),
    ChatMessage(role="user", content="What is your name"),
]

resp = AI21(api_key=api_key).chat(
    messages, preamble_override="You are a pirate with a colorful personality"
)

In [ ]:
print(resp)

#### Call `complete` with a prompt

In [ ]:
from llama_index.llms.ai21 import AI21

api_key = "Your api key"
resp = AI21(api_key=api_key).complete("Paul Graham is ")

In [ ]:
print(resp)


an American computer scientist, essayist, and venture capitalist. He is best known for his work on Lisp, programming language design, and entrepreneurship. Graham has written several books on these topics, including " ANSI Common Lisp" and " Hackers and Painters." He is also the co-founder of Y Combinator, a venture capital firm that invests in early-stage technology companies.


## Configure Model

Configure the parameter passed to the model like - `max_tokens`, `temperature`

In [ ]:
from llama_index.llms.ai21 import AI21

llm = AI21(
    model="jamba-instruct", api_key=api_key, max_tokens=100, temperature=0.5
)

In [ ]:
resp = llm.complete("Paul Graham is ")

In [ ]:
print(resp)


an American computer scientist, essayist, and venture capitalist. He is best known for his work on Lisp, programming language design, and entrepreneurship. Graham has written several books on these topics, including " ANSI Common Lisp" and " Hackers and Painters." He is also the co-founder of Y Combinator, a venture capital firm that invests in early-stage technology companies.


## Set API Key at a per-instance level
If desired, you can have separate LLM instances use separate API keys.

In [ ]:
from llama_index.llms.ai21 import AI21

llm_good = AI21(api_key=api_key)
llm_bad = AI21(model="j2-mid", api_key="BAD_KEY")

resp = llm_good.complete("Paul Graham is ")
print(resp)

resp = llm_bad.complete("Paul Graham is ")
print(resp)

## Streaming

using `stream_chat` method

In [ ]:
from llama_index.llms.ai21 import AI21
from llama_index.core.llms import ChatMessage

llm = AI21(api_key=api_key, model="jamba-instruct")
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="Tell me a story"),
]
resp = llm.stream_chat(messages)

In [ ]:
for r in resp:
    print(r.delta, end="")

None Once upon a time, there was a pirate named Captain Jack. He was a very colorful pirate with a love for adventure and treasure. He sailed the seven seas, always searching for his next big score.

One day, he stumbled upon a map that led to a hidden island filled with gold and riches beyond his wildest dreams. He gathered his loyal crew and set sail towards the island.

As they approached the island, they were met with treacherous waters and dangerous obstacles. But Captain Jack was determined to find the treasure, and he wouldn't let anything stand in his way.

Finally, after days of searching, they found the hidden island. They docked their ship and made their way through the dense jungle, following the map to the treasure.

As they approached the treasure, they were met with a group of fierce warriors who guarded the treasure. But Captain Jack and his crew were not afraid. They fought bravely and defeated the warriors, finally reaching the treasure.

They opened the treasure ches

## Tokenizer

The type of the tokenizer is determined by the name of the model 

In [ ]:
from llama_index.llms.ai21 import AI21

llm = AI21(api_key=api_key, model="jamba-instruct")

tokenizer = llm.tokenizer

tokens = tokenizer.encode("Hello llama-index!")

decoded = tokenizer.decode(tokens)

print(decoded)